In [2]:
import numpy as np
import pandas as pd
import ipywidgets as ipw
import matplotlib.pyplot as plt
import vpython as vp

from scipy.linalg import expm

from random import choice

from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, ColorBar, BasicTickFormatter, BasicTicker, FixedTicker, FuncTickFormatter
output_notebook()

<IPython.core.display.Javascript object>

Loading BokehJS ...

In [3]:
N = 10; # Define the lattice size
#lattice = np.empty((N*N,3),int); # Create a container for the lattice sites
Apos = [None]*(N*N); # Create a container for site positions using vpython.vector()
#spin = np.ones(N*N); # Create a container for the spins for all the lattice sites
Vspin = [None]*(N*N);
for ii in range(N*N):
    #lattice[ii] = [ii%N, np.floor((ii)/N), 0];
    Apos[ii] = vp.vector(ii%N, np.floor((ii)/N), 0);
    Vspin[ii] = vp.vector(1,0,0);
for ii in range(N):
    kk = choice(range(N*N));
    Vspin[kk] = vp.vector(1,1,1);

In [49]:
# Define a function to map a 1D array to a 2D/3D lattice, only works on lattice of equal-lateral size in Cartesian system
def map(index, size, dimension): 
    if dimension == 1:
        return index;
    if dimension == 2:
        x = index%size;
        y = np.floor(index/size);
        return vp.vector(x,y,0);
    if dimension == 3:
        sub_index = index%(size**2);
        x = sub_index%size;
        y = np.floor(sub_index/size);
        z = np.floor(index/size**2);
        return vp.vector(x,y,z)

In [50]:
def antimap(vector, size, dimension):
    if dimension == 1:
        return vector.x;
    if dimension == 2:
        return vector.x + vector.y*size;
    if dimension == 3:
        return vector.x + vector.y*size + vector.z*size*size;

In [36]:
# Define a function to find the index of the nearest neighbours
def nNbr(index, size, dimension):
    if dimension == 1:
        return [index-1, index+1];
    
    if dimension == 2:
        pos = map(index,size,dimension);
        north = pos + vp.vector(0,1,0);
        south = pos + vp.vector(0,-1,0);
        east = pos + vp.vector(-1,0,0);
        west = pos + vp.vector(1,0,0);
        return [antimap(north, size, 2), antimap(south, size, 2), antimap(east, size, 2), antimap(west, size, 2)];
    
    if dimension == 3:
        north = pos + vp.vector(0,1,0);
        south = pos + vp.vector(0,-1,0);
        east = pos + vp.vector(-1,0,0);
        west = pos + vp.vector(1,0,0);
        up = pos + vp.vector(0,0,1);
        down = pos + vp.vector(1,0,0);
        return [antimap(north, size, 3), antimap(south, size, 3), antimap(east, size, 3), antimap(west, size, 3), antimap(up, size, 3), antimap(down, size, 3)];
    

In [53]:
test = map(212,10,3);
test

<2, 1, 2>

In [55]:
antimap(test,10,3)

212.0

In [ ]:
r = 0.2; # dot radius for visualization
for ii in range(N*N):
    vp.sphere(pos=Apos[ii], radius=r, color=Vspin[ii])